In [1]:
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option("display.max_columns",None)
pd.set_option("display.float_format",lambda x: '%.3f' % x)

df_ = pd.read_excel("online_retail_II.xlsx",sheet_name="Year 2009-2010")
df_.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [13]:
df = df_.copy()

In [14]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [15]:
df.dropna(inplace=True)

In [18]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,417534.000,417534,417534.000,417534.000
mean,12.759,2010-07-01 01:12:19.787609856,3.888,15360.645
min,-9360.000,2009-12-01 07:45:00,0.000,12346.000
25%,2.000,2010-03-26 11:26:00,1.250,13983.000
50%,4.000,2010-07-08 19:12:00,1.950,15311.000
75%,12.000,2010-10-14 14:08:00,3.750,16799.000
max,19152.000,2010-12-09 20:01:00,25111.090,18287.000
std,101.220,NaN,71.132,1680.811


In [20]:
df[df["Invoice"].str.contains("C",na=False)]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.950,16321.000,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.650,16321.000,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.250,16321.000,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.100,16321.000,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.950,16321.000,Australia
...,...,...,...,...,...,...,...,...
524695,C538123,22956,36 FOIL HEART CAKE CASES,-2,2010-12-09 15:41:00,2.100,12605.000,Germany
524696,C538124,M,Manual,-4,2010-12-09 15:43:00,0.500,15329.000,United Kingdom
524697,C538124,22699,ROSES REGENCY TEACUP AND SAUCER,-1,2010-12-09 15:43:00,2.950,15329.000,United Kingdom
524698,C538124,22423,REGENCY CAKESTAND 3 TIER,-1,2010-12-09 15:43:00,12.750,15329.000,United Kingdom


In [21]:
df=df[~df["Invoice"].str.contains("C",na=False)]

In [23]:
df["Description"].value_counts()

Description
WHITE HANGING HEART T-LIGHT HOLDER    3153
REGENCY CAKESTAND 3 TIER              1706
STRAWBERRY CERAMIC TRINKET BOX        1407
ASSORTED COLOUR BIRD ORNAMENT         1369
HOME BUILDING BLOCK WORD              1219
                                      ... 
BAROQUE BUTTERFLY EARRINGS RED           1
BAROQUE BUTTERFLY EARRINGS CRYSTAL       1
BLACK DROP EARRINGS W LONG BEADS         1
BLACK DIAMOND CLUSTER EARRINGS           1
BAKING MOULD EASTER EGG MILK CHOC        1
Name: count, Length: 4444, dtype: int64

In [24]:
df.groupby("Description")["Quantity"].sum().sort_values(ascending =False)

Description
WHITE HANGING HEART T-LIGHT HOLDER    56915
WORLD WAR 2 GLIDERS ASSTD DESIGNS     54754
BROCADE RING PURSE                    48166
PACK OF 72 RETRO SPOT CAKE CASES      45156
ASSORTED COLOUR BIRD ORNAMENT         44551
                                      ...  
BAROQUE BUTTERFLY EARRINGS CRYSTAL        1
OPAL WHITE/SILVER FLOWER NECKLACE         1
WHITE CHRISTMAS TREE 60CM                 1
SILICON CUBE 25W, BLUE                    1
DROP EARRINGS W FLOWER & LEAF             1
Name: Quantity, Length: 4444, dtype: int64

In [25]:
df["total_price"] = df["Quantity"] * df["Price"]

In [30]:
df.groupby("Invoice").agg({"total_price":"sum"})

,total_price
Invoice,
489434,505.300
489435,145.800
489436,630.330
489437,310.750
489438,2286.240
...,...
538167,298.950
538168,318.700
538169,93.450


In [31]:
df["InvoiceDate"].max()

Timestamp('2010-12-09 20:01:00')

In [34]:
today_date = dt.datetime(2010,12,11)
(today_date - df["InvoiceDate"].min()).days

374

In [37]:
rfm = df.groupby("Customer ID").agg({"InvoiceDate":lambda InvoiceDate: (today_date-InvoiceDate.max()).days,
                                     "Invoice":lambda invoice: invoice.nunique(),
                                     "total_price":lambda total_price:total_price.sum()})
rfm.index = rfm.index.astype(int);rfm

,InvoiceDate,Invoice,total_price
Customer ID,,,
12346,165,11,372.860
12347,3,2,1323.320
12348,74,1,222.160
12349,43,3,2671.140
12351,11,1,300.930
...,...,...,...
18283,18,6,641.770
18284,67,1,461.680
18285,296,1,427.000


In [38]:
rfm.columns = ["recency","frequency","monetary"]
rfm.describe()

,recency,frequency,monetary
count,4314.000,4314.000,4314.000
mean,91.270,4.454,2047.289
std,96.944,8.169,8912.523
min,1.000,1.000,0.000
25%,18.000,1.000,307.950
50%,53.000,2.000,705.550
75%,136.000,5.000,1722.802
max,374.000,205.000,349164.350


In [41]:
rfm = rfm[rfm["monetary"]>0]
rfm.shape

(4312, 3)

In [42]:
rfm["recency_score"] = pd.qcut(rfm["recency"],5,[5,4,3,2,1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"),5,[1,2,3,4,5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"],5,[1,2,3,4,5])
rfm

,recency,frequency,monetary,recency_score,frequency_score,monetary_score
Customer ID,,,,,,
12346,165,11,372.860,2,5,2
12347,3,2,1323.320,5,2,4
12348,74,1,222.160,2,1,1
12349,43,3,2671.140,3,3,5
12351,11,1,300.930,5,1,2
...,...,...,...,...,...,...
18283,18,6,641.770,4,5,3
18284,67,1,461.680,3,2,2
18285,296,1,427.000,1,2,2


In [43]:
rfm["rfm_score"] = rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str)
rfm

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,rfm_score
Customer ID,,,,,,,
12346,165,11,372.860,2,5,2,25
12347,3,2,1323.320,5,2,4,52
12348,74,1,222.160,2,1,1,21
12349,43,3,2671.140,3,3,5,33
12351,11,1,300.930,5,1,2,51
...,...,...,...,...,...,...,...
18283,18,6,641.770,4,5,3,45
18284,67,1,461.680,3,2,2,32
18285,296,1,427.000,1,2,2,12


In [44]:
seg_map = {
    r'[1-2][1-2]':"hibernating",
    r'[1-2][3-4]':"at_risk",
    r'[1-2]5':"cant_loose",
    r'3[1-2]':"about_to_sleep",
    r'33':"need_attention",
    r'[3-4][4-5]':"loyal_customers",
    r'41':"promising",
    r'51':"new_customers",
    r'[4-5][2-3]':"potential_loyalists",
    r'5[4-5]':"champions"
}

rfm["segment"] = rfm["rfm_score"].replace(seg_map,regex=True)
rfm

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,rfm_score,segment
Customer ID,,,,,,,,
12346,165,11,372.860,2,5,2,25,cant_loose
12347,3,2,1323.320,5,2,4,52,potential_loyalists
12348,74,1,222.160,2,1,1,21,hibernating
12349,43,3,2671.140,3,3,5,33,need_attention
12351,11,1,300.930,5,1,2,51,new_customers
...,...,...,...,...,...,...,...,...
18283,18,6,641.770,4,5,3,45,loyal_customers
18284,67,1,461.680,3,2,2,32,about_to_sleep
18285,296,1,427.000,1,2,2,12,hibernating


In [45]:
rfm.groupby("segment")[["recency","frequency","monetary"]].agg(["mean","count"])

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       53.819   343     1.201   343  441.320   343
at_risk             152.159   611     3.074   611 1188.878   611
cant_loose          124.117    77     9.117    77 4099.450    77
champions             7.119   663    12.554   663 6852.264   663
hibernating         213.886  1015     1.126  1015  403.978  1015
loyal_customers      36.287   742     6.830   742 2746.067   742
need_attention       53.266   207     2.449   207 1060.357   207
new_customers         8.580    50     1.000    50  386.199    50
potential_loyalists  18.793   517     2.017   517  729.511   517
promising            25.747    87     1.000    87  367.087    87

In [51]:
rfm[rfm["segment"]=="cant_loose"].index

Index([12346, 12380, 12482, 12510, 12891, 12932, 13044, 13313, 13680, 13782,
       13799, 13856, 14025, 14063, 14160, 14221, 14548, 14607, 14685, 14745,
       15003, 15013, 15015, 15125, 15141, 15222, 15306, 15321, 15359, 15369,
       15372, 15443, 15538, 15607, 15633, 15722, 15751, 15754, 15768, 15911,
       15912, 16027, 16032, 16158, 16177, 16197, 16335, 16467, 16631, 16742,
       16743, 16875, 16986, 17021, 17032, 17092, 17113, 17157, 17188, 17230,
       17268, 17426, 17448, 17454, 17512, 17578, 17602, 17651, 17940, 17969,
       17988, 18009, 18051, 18064, 18094, 18251, 18258],
      dtype='int32', name='Customer ID')

In [54]:
new_df = pd.DataFrame()
new_df["new_customers_id"] = rfm[rfm["segment"]=="new_customers"].index
new_df.head()

,new_customers_id
0,12351
1,12385
2,12386
3,12427
4,12441


In [55]:
new_df.to_csv("new_customers.csv")
rfm.to_csv("rfm.csv")

In [64]:
df = df_.copy()

def create_rfm(df,csv=False):
    #data prep
    df.dropna(inplace=True)
    df = df[~df["Invoice"].str.contains("C",na=False)]
    df["total_price"] = df["Quantity"] * df["Price"]
    
    #rfm metrics
    today_date = dt.datetime(2011,12,11)
    rfm = df.groupby("Customer ID").agg({"InvoiceDate":lambda InvoiceDate: (today_date- InvoiceDate.max()).days,
                                        "Invoice": lambda invoice: invoice.nunique(),
                                        "total_price":lambda total_price:total_price.sum()})
    rfm.index = rfm.index.astype(int)
    rfm.columns = ["recency","frequency","monetary"]
    rfm = rfm[rfm["monetary"]>0]    
    
    #rfm scores
    rfm["recency_score"] = pd.qcut(rfm["recency"],5,labels=[5,4,3,2,1])
    rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"),5,[1,2,3,4,5])
    rfm["monetary_score"]  = pd.qcut(rfm["monetary"],5,labels=[1,2,3,4,5])
    
    rfm["rfm_score"] = rfm["recency_score"].astype(str)  + rfm["frequency_score"].astype(str)  
    
    #segments
    seg_map = {
    r'[1-2][1-2]':"hibernating",
    r'[1-2][3-4]':"at_risk",
    r'[1-2]5':"cant_loose",
    r'3[1-2]':"about_to_sleep",
    r'33':"need_attention",
    r'[3-4][4-5]':"loyal_customers",
    r'41':"promising",
    r'51':"new_customers",
    r'[4-5][2-3]':"potential_loyalists",
    r'5[4-5]':"champions"
    }

    rfm["segment"] = rfm["rfm_score"].replace(seg_map,regex=True)

    rfm= rfm[["recency","frequency","monetary","segment"]]
    
    
    if csv:
        rfm.to_csv("rfm.csv")
        
    return rfm

In [ ]:
rfm=create_rfm(df,csv=True)

In [69]:
rfm

,recency,frequency,monetary,segment
Customer ID,,,,
12346,530,11,372.860,cant_loose
12347,368,2,1323.320,potential_loyalists
12348,439,1,222.160,hibernating
12349,408,3,2671.140,need_attention
12351,376,1,300.930,new_customers
...,...,...,...,...
18283,383,6,641.770,loyal_customers
18284,432,1,461.680,about_to_sleep
18285,661,1,427.000,hibernating
